## **셀레니움 활용한 네이버 엔드라이버 사진 다운**

In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver

chrome_executable_path = '/path/to/google-chrome'
chromedriver_path = '/path/to/chromedriver'
options = webdriver.ChromeOptions()
options.binary_location = chrome_executable_path

driver = webdriver.Chrome(chrome_driver_binary=chromedriver_path, options=options)
driver.get('https://nid.naver.com/nidlogin.login?url=https%3A%2F%2Fmybox.naver.com')


In [ ]:
#본인의 개인정보를 입력하세요
myid = '*'
mypw = '*'

In [ ]:
#아이디 복사
ele= driver.find_element(by='id',value='id')
ele.click()
pyperclip.copy(myid)
ele.send_keys(Keys.CONTROL,'v')


In [ ]:
#비밀번호 복사
ele= driver.find_element(by='id',value='pw')
ele.click()
pyperclip.copy(mypw)
ele.send_keys(Keys.CONTROL,'v')

In [ ]:
#로그인
elems=driver.find_elements(By.CSS_SELECTOR,'#log\.login')
elems[0].click()

In [ ]:
elems=driver.find_elements(By.CSS_SELECTOR,'#nav_lnb > div.nav_lnb_scroll > div.lnb_area > div:nth-child(2) > ul > li:nth-child(1) > div > ul > li:nth-child(1) > a')
elems[0].click()


In [ ]:
elems=driver.find_elements(By.CSS_SELECTOR,'#content > div.content_box > div > div > div > div.all_photo_list_area.year > div:nth-child(1) > h4 > label')
elems[0].click()


In [ ]:
elems=driver.find_elements(By.CSS_SELECTOR,'#content > div.task_area_wrap.no_line > div > div.task_top_area.top_btn > div > button:nth-child(3)')
elems[0].click()


In [ ]:
import os
import datetime
import zipfile

# 다운로드된 파일의 경로
downloaded_zip_path = '/Users/user/Downloads/MYBOX.zip'  # 실제 다운로드된 zip 파일의 경로로 변경하세요
extracted_folder_path = '/Users/user/Desktop/엔드라이브'  # 압축 해제된 파일들을 저장할 폴더 경로로 변경하세요

import os


# 폴더 생성
folder_name = 'python_exam'  # 현재 날짜와 시간을 이용한 폴더 이름 생성
folder_path = f'/Users/user/Desktop/{folder_name}'  # 원하는 경로로 변경 가능
os.makedirs(folder_path, exist_ok=True)  # 폴더 생성 (이미 폴더가 존재하는 경우에도 덮어쓰지 않음)

# 압축 해제
with zipfile.ZipFile(downloaded_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# 압축 해제된 파일들 이동
extracted_files = os.listdir(extracted_folder_path)
for file in extracted_files:
        file_path = os.path.join(extracted_folder_path, file)
        new_file_path = os.path.join(folder_path, file)
        os.rename(file_path, new_file_path)


# 생성된 폴더에 접근하여 PIL 라이브러리를 활용한 이미지 정보 (위치, 날짜 ) 추출

In [ ]:
import os

folder_path = '/content/drive/MyDrive/Colab Notebooks/python_exam'  # 이미지가 저장된 폴더 경로로 변경하세요

# 이미지 파일들의 경로 가져오기
image_files = [file for file in os.listdir(folder_path) if file.endswith('.JPG')]

image_paths = []  # 이미지 파일의 경로를 저장할 리스트

for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    image_paths.append(image_path) # 이미지 파일 경로를 리스트에 저장

# 이미지 파일의 경로 출력
for image_path in image_paths:
    print(f"이미지 경로: {image_path}")


In [ ]:
import os
from PIL import Image ,UnidentifiedImageError
from PIL.ExifTags import TAGS
from fractions import Fraction

def extract_exif_data(image_path):
    # 이미지 열기
    image = Image.open(image_path)

    # 이미지의 EXIF 데이터 가져오기
    exif_data = image._getexif()
    if exif_data is None:
        return None

    # 추출한 EXIF 데이터를 저장할 딕셔너리
    exif_info = {}
    lat_list=[]
    lon_list=[]
    # EXIF 데이터를 순회하며 필요한 정보 추출
    for tag_id, value in exif_data.items():
        tag_name = TAGS.get(tag_id, tag_id)
        exif_info[tag_name] = value

    return exif_info

def extract_location_and_date(folder_path):
    # 폴더 내의 파일들을 확인하고 EXIF 데이터를 추출하여 위치와 날짜 정보를 저장할 딕셔너리
    image_info = {}

    for filename in os.listdir(folder_path):
        # 파일의 전체 경로 생성
        file_path = os.path.join(folder_path, filename)

        # 이미지 파일인지 확인
        try:
            img = Image.open(file_path)
        except UnidentifiedImageError:
            continue

        exif_info = extract_exif_data(file_path)

    # Initialize latitude and longitude variables
        latitude = None
        longitude = None

        # EXIF 데이터에서 위치 정보와 날짜 정보 추출
        if exif_info is not None:
            lat = exif_info.get('GPSInfo', {}).get(2)
            lon = exif_info.get('GPSInfo', {}).get(4)

            if lat is not None:
                latitude = float(Fraction(lat[0])) + float(Fraction(lat[1])) / 60 + float(Fraction(lat[2])) / 3600
            if lon is not None:
                longitude = float(Fraction(lon[0])) + float(Fraction(lon[1])) / 60 + float(Fraction(lon[2])) / 3600

            date = exif_info.get('DateTimeOriginal')
            # 위치 정보와 날짜 정보를 딕셔너리에 저장
            image_info[file_path] = {'Latitude': latitude, 'Longitude': longitude, 'Date': date}
    return image_info

# 이미지 파일들이 있는 폴더 경로
folder_path = '/content/drive/MyDrive/Colab Notebooks/python_exam'

# 위치와 날짜 정보 추출
image_info = extract_location_and_date(folder_path)

# 추출한 정보 출력
for image_path, info in image_info.items():
    print('Image:', image_path)

    print('Latitude:', info['Latitude'])
    print('Longitude:', info['Longitude'])
    print('Date:', info['Date'])
    print('---')


# 받아온 이미지정보를 활용하여 시각화에 활용될 위치, 날짜, 이미지사진, 이미지분류를 나타내는 코드를 작성하고 html 로 저장

In [ ]:
import folium
from PIL import Image
import base64
from datetime import datetime
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim
from io import BytesIO
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image as keras_image

def load_mobilenetv2_model():
    model = MobileNetV2(weights='imagenet')
    return model

def classify_image(model, img):
    img = img.resize((224, 224))
    img_array = keras_image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    preds = model.predict(img_array)
    label = decode_predictions(preds, top=1)[0][0][1].lower()
    return label

# Load the model
mobilenet_v2_model = load_mobilenetv2_model()
def get_icon_by_class(image_class):
    # 음식 관련 이미지 클래스
    food_keywords = ["food", "drink", "coffee", "fruit", "hot_pot", "plate", "pot", "cleaver","cup"]
    # 풍경 관련 이미지 클래스
    landscape_keywords = ["landscape", "mountain", "nature", "patio"]
    # 사람 관련 이미지 클래스
    people_keywords = ["person", "face", "portrait", "user", "groom", "lab_coat", "ski_mask", "trench_coat", "sweatshirt", "suit","monitor","ski_mask","fur_coat"]
    # 이동수단 관련 이미지 클래스
    transportation_keywords = ["car", "bus", "train", "bicycle", "motorbike", "airplane", "ship", "seat_belt"]

    if any(keyword in image_class for keyword in food_keywords):
        icon_name = "cutlery"
    elif any(keyword in image_class for keyword in landscape_keywords):
        icon_name = "tree"
    elif any(keyword in image_class for keyword in people_keywords):
        icon_name = "user"
    elif any(keyword in image_class for keyword in transportation_keywords):
        icon_name = "car"
    else:
        icon_name = "compass"
    return icon_name


mobilenet_v2_model = load_mobilenetv2_model()


def img_to_base64(img):
    buffered = BytesIO()
    img.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode()


def find_centroid(coords):
    x_coords = [point[0] for point in coords]
    y_coords = [point[1] for point in coords]
    centroid = (sum(x_coords) / len(coords), sum(y_coords) / len(coords))
    return centroid

grouped_images = {}
for image_path, info in image_info.items():
    if info['Date'] is not None:
        date_key = datetime.strptime(info['Date'], '%Y:%m:%d %H:%M:%S').date()

    if date_key in grouped_images:
        grouped_images[date_key].append((image_path, info))
    else:
        grouped_images[date_key] = [(image_path, info)]

date_list = list(grouped_images.keys())

# 날짜 목록 출력
print("날짜 목록:")
for index, date in enumerate(date_list):
    image_count = len(grouped_images[date])   # Add this line
    print(f"{index + 1}. {date} - 이미지 개수: {image_count}")   # Modified line

# 날짜 선택 입력
selected_index = int(input("날짜를 선택하세요 (번호 입력): ")) - 1
selected_date = date_list[selected_index]

initial_coords = [x for x in [(image[1]['Latitude'], image[1]['Longitude']) for image in grouped_images[selected_date]] if x[0] is not None and x[1] is not None]


if not initial_coords:
    print("선택한 날짜에 위치 정보를 포함한 이미지가 없습니다.")  # Notify the user that there are no images with location information
else:
    initial_coords = initial_coords[0]

    geolocator = Nominatim(user_agent='my_geocoder')
    # The rest of the code related to folium should be indented one more level to the right within the else block

    sorted_images = sorted([x for x in grouped_images[selected_date] if x[1]['Date'] is not None], key=lambda x: datetime.strptime(x[1]['Date'], "%Y:%m:%d %H:%M:%S"))

    m = folium.Map(location=initial_coords, zoom_start=15)
    marker_cluster = MarkerCluster().add_to(m)
    cluster_coords = []
    # The rest of your code

lines = []


# MarkerCluster를 담을 딕셔너리 생성
marker_clusters = {
    'food': MarkerCluster(name="Food").add_to(m),
    'landscape': MarkerCluster(name="Landscape").add_to(m),
    'person': MarkerCluster(name="Person").add_to(m),
    'transport' : MarkerCluster(name="Transport").add_to(m)
}

for index, (path, info) in enumerate(sorted_images):
    latitude, longitude = info.get('Latitude'), info.get('Longitude')

    if latitude is None or longitude is None:
        continue

    img = Image.open(path).convert("RGB")
    img.thumbnail((300, 300))
    cluster_coords.append((latitude, longitude))
    location = geolocator.reverse(f"{latitude}, {longitude}")

    if index > 0:
        prev_info = sorted_images[index - 1][1]
        prev_latitude, prev_longitude = prev_info.get('Latitude'), prev_info.get('Longitude')

        if prev_latitude is not None and prev_longitude is not None:
            lines.append(((prev_latitude, prev_longitude), (latitude, longitude)))

        html = f"""
        <html>
        <head>
        <style>
        .popup-img {{
          max-height: 300px;
          max-width: 300px;
          width: auto;
          height: auto;
        }}
        </style>
        </head>
        <body>
        <div>
        <p>날짜: {selected_date}</p>
        <p>위치: {location.address}</p>
        <img src="data:image/jpeg;base64,{img_to_base64(img)}" alt="Image not found" style="display:block;margin:auto" />
        </div>
        </body>
        </html>
        """
    image_class = classify_image(mobilenet_v2_model, img)
    print(f"Image class: {image_class}")   # Add this line
    icon_name = get_icon_by_class(image_class)

    cluster_name = None
    if icon_name == 'cutlery':
        cluster_name = 'food'
    elif icon_name == 'compass':
        cluster_name = 'landscape'
    elif icon_name == 'user':
        cluster_name = 'person'
    elif icon_name == 'car':
        cluster_name = 'transport'


    icon = folium.Icon(icon=icon_name, prefix='fa')
    iframe = folium.IFrame(html, width=500, height=500)
    popup = folium.Popup(iframe, max_width=2000)
    if cluster_name is not None:
        marker_cluster = marker_clusters[cluster_name]
    else:
        marker_cluster = folium.map.FeatureGroup()  # 기본 레이어에 마커만 추가하게 됩니다.

    folium.Marker(location=[latitude, longitude], icon=icon, popup=popup).add_to(marker_cluster)

if len(cluster_coords) > 0:
    centroid = find_centroid(cluster_coords)
    folium.Marker(location=centroid, icon=folium.Icon(color='red')).add_to(m)

# 선 추가
for line in lines:
    folium.PolyLine(line, color='red', weight=5).add_to(m)

folium.LayerControl().add_to(m)
m.save(f"map_{selected_date}.html")
m
